In [11]:
import numpy as np
import mediapipe as mp
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
from tqdm import tqdm  # ✅ tqdm for progress bar

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
path = "./face_ds"
categories = ['angry', 'happy', 'neutral', 'sad', 'surprise']

features = []
labels = []

# ✅ Prepare data with tqdm progress bar
for category_index, category in enumerate(categories):
    new_path = os.path.join(path, category)
    for file in tqdm(os.listdir(new_path), desc=f"Processing {category}"):
        img = cv2.imread(os.path.join(new_path, file))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,(224,224))
        result = face_mesh.process(img)
        if result.multi_face_landmarks:
            face_landmarks = result.multi_face_landmarks[0]
            landmarks = []
            for lm in face_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            if len(landmarks) == 1404:
                features.append(landmarks)
                labels.append(category_index)

# split train and test
features = np.asarray(features)
labels = np.asarray(labels)

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, stratify=labels)

# train model
model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
model.fit(x_train, y_train)

# test model
y_pred = model.predict(x_test)

# accuracy score and confusion matrix
print(f"The accuracy score is: {accuracy_score(y_pred, y_test)}")
print("Confusion matrix:")
print(confusion_matrix(y_pred, y_test))

# save model
with open("model_face.pkl", "wb") as f:
    pickle.dump(model, f)


I0000 00:00:1750434605.069795   28556 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1750434605.083111   29490 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
W0000 00:00:1750434605.134736   29486 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750434605.220656   29485 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing surprise: 100%|████████████████████| 250/250 [00:09<00:00, 25.32it/s]
/home/mehdi-zain/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:50:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


The accuracy score is: 0.5521739130434783
Confusion matrix:
[[19  5  5  9  2]
 [ 4 26  0  3  6]
 [ 7  1 32 13  5]
 [ 8  7 12 20  3]
 [ 8  4  0  1 30]]


In [10]:
import xgboost as xg